In [1]:
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt

# Objectives

1. **Given a linear or nonlinear system, choose an appropriate numerical solution method based on the properties of the system**: 

2. **Evaluate a method for its convergence and computational cost, including parallel computing aspects**

3. **Diagnose convergence problems of iterative solution methods**

4. **Select or design a method or approach for preconditioning the solution of specific problems**

5. **Use [scipy], [PETSc], [petsc] or other numerical software for solving systems of equations**


## Model Problem

We explore the theoretical and empirical performance of the proposed Jacobian-free Newtons method using preconditioned Krylov subspace methods for solving nonlinear optimization problem based on [this paper](https://epubs.siam.org/doi/abs/10.1137/0905039?casa_token=5o2ebQDJ-P0AAAAA:FzkM1jsrCALfpTZ9jA6gU4EOiWlZ1RRWVV8UbhrZ_5Kylnw4zHiiY-Bh1p8Iw_lhlT2j0Zvjng) and compare it with other exact and approximate Jacobian-based methods. Our first model problem is the 1-D nonlinear PDE,

$$-u_{xx} + 2b(e^u)_x + ce^u = R(x) \text{ on the domain } 0 < x < 1$$

with homogeneous Dirichlet boundary conditions, or $u_0=u_{n+1}=0$. To numerically solve this, we discretize it on a uniform gird over $n$ itervals using centered finite differencing. We write its formulation below.

In [2]:
def nonlinear_PDE(u):
    """ Centered finite difference of nonlinear PDE
            -u_xx + 2b(e^u)_x + ce^u = R(x) for x \in [0,1]
        with homogenous Dirichlet boundary conditions.
    """
    n = len(u)
    h = 1 / (n + 1)
    # TODO: allow @b,@c to be parameters
    b = c = 1

    expu = np.exp(u)

    r1 = (1/h**2) * (sp.diags([1, -2, 1], [-1, 0, 1], shape=(n, n)).dot(u))
    r2 = (1/(2 * h)) * (sp.diags([-1, 0, 1], [-1, 0, 1], shape=(n, n)).dot(expu))
    r3 = expu

    r = -r1 + 2*b*r2 + c*r3
    return r

In [3]:
# TODO: Rafael Includes 2nd Model Problem If He Gets the Chance

## Newton's Method

Given a nonlinear objective function $F$ and its Jacobian $F_J$, where we want to find $x^\star$ such that $F(x^\star) = 0$, we can find the solution using Newton's method. The template code is as follows:

1. Start with an initial guess $x_0$
2. Repeat until convergence
    1. Solve $J_F(x_i)s = -F(x_i)$
    2. Set $x_{i+1} \leftarrow x_i + s$
    
For our problem at hand, we want to find a solution $u$ that satisfies the PDEs described above. Therefore, we can model our problem as

$$F(u) = PDE(u) - R(x)$$

where $PDE(u)$ is the finite-differnce PDE evaluated with solution $u$ (as formulated above) and $R(x)$ is the true solution of the PDE. Now, we need the Jacobian of system from the finite differencing of the PDE. This can be derivied either analytically or by using finite differencing once more. We include both codes below.

In [6]:
def exact_jacobian(F, u):
    """ Exact Jacobian of finite difference model PDE from above """
    n = len(u)
    h = 1 / (n + 1)
    # TODO: allow @b,@c to be parameters
    b = c = 1
    expu = np.exp(u)

    subdiag = -(1 / h**2) - 2 * b / (2 * h) * expu[:n - 1]
    maindiag = 2 / (h**2) + c * expu
    supdiag = -(1 / h**2) + 2 * b / (2 * h) * expu[1:]

    J = sp.diags(subdiag, -1) + sp.diags(maindiag, 0) + sp.diags(supdiag, 1)

    return J

def fd_jacobian(F, u):
    """ Finite Difference Approximation of Jacobian for any function F.

    - F : Function eval. Returns 1d np.ndarray
    - x : Current solution

    Returns: @J csr_matrix
    """
    n = len(u)
    J = np.zeros((n, n))
    slen = 1e-4
    Fu = F(u)

    for i in range(n):
        e_i = np.append(np.zeros(i), np.append(1, np.zeros(n - i - 1)))
        # TODO: Drop terms with small values to induce sparsity?
        J[:, i] = (F(u + slen * e_i) - Fu) / slen

    return sp.csr_matrix(J)

Before we write the code for Newton's method with a matrix of the Jacobian, we observe that in each iteration of Newton's method we need to solve the linear system $J_F(x_i)s = -F(x_i)$. We consider three methods for solving this:

- Sparse direct solver
- GMRES
- BiCG? CGS?

We recall the convergence of GMRES for solving $Ax=b$ depends on the residual bound. Assuming the matrix $A$ is diagonalizable via $A=X^{-1}\Lambda X$ where $\Lambda=\mathrm{diag}(\lambda_1,\ldots,\lambda_n)$, then we at the $m$ iteration that

$$ \frac{\|r_m \|_2}{\|r_0\|_2} \leq \kappa_2(X) \min_{p \in \mathbb{P}_m, p(0)=1} \max_{i=1,\ldots,n} |p(\lambda_i)|$$

In other words, the non-normality of the Jacobian matrix $J$ can impact the convergence when using GMRES. Below, we examine $\kappa(X)$ for the exact and finite-difference Jacobian matrix with different solutions for $u$. 

In [25]:
np.random.seed(0) # fix randomness 

# which solution @u we use
for i in range(2):
    # exact or FD jacobian
    for j in range(2):
        # size of problem
        for k in range(1,4):
            
            n = 10**k
            
            if(i==0): u = np.ones(n)
            else: u = 10*np.random.random(n)
                
            if(j==0): J = exact_jacobian(nonlinear_PDE, u)
            else: J = fd_jacobian(nonlinear_PDE, u)
                
            _,X = la.eig(J.todense())
            condX = la.cond(X)
            
            print("Cond(X) for {} Jacobian of size n={:<4} with {} solution => {:.4e}".format(\
                    "exact" if j==0 else "FD",
                    n,
                    "uniform" if i==0 else "random",
                    condX))
            
        print("===")

Cond(X) for exact Jacobian of size n=10   with uniform solution => 9.7716e+00
Cond(X) for exact Jacobian of size n=100  with uniform solution => 1.4370e+01
Cond(X) for exact Jacobian of size n=1000 with uniform solution => 1.5072e+01
===
Cond(X) for FD Jacobian of size n=10   with uniform solution => 9.7727e+00
Cond(X) for FD Jacobian of size n=100  with uniform solution => 1.4372e+01
Cond(X) for FD Jacobian of size n=1000 with uniform solution => 1.5074e+01
===
Cond(X) for exact Jacobian of size n=10   with random solution => 3.9633e+00
Cond(X) for exact Jacobian of size n=100  with random solution => 3.3224e+08
Cond(X) for exact Jacobian of size n=1000 with random solution => 2.3783e+16
===
Cond(X) for FD Jacobian of size n=10   with random solution => 5.1872e+01
Cond(X) for FD Jacobian of size n=100  with random solution => 2.9460e+09
Cond(X) for FD Jacobian of size n=1000 with random solution => 2.7368e+16
===


From here, we see that the choice of exact or finite difference Jacobian does not make much of a difference theoretically. However, the true solution $u^\star$ and any approximate solution $u$ encountered during the Newton's step can severely hamper the convergence of GMRES potentially. 

With this in mind, we move onto Newton's method.

In [24]:
def newtons(F, J, x0, tol=1e-6):
    """ Newton's Method with exact or FD Jacobian

    - F : Function eval. Returns 1d np.ndarray
    - J : Jacobian eval. Returns 2d np.ndarray
    - x0 : Initial guess. 1D np.ndarray

    Returns:
    - @x solution so that F(x)=0
    - @numiters, int
    - @score_hist, history of "F(x)". 1d np.ndarray
    """
    x = x0
    niters = 0
    score_hist = np.array([la.norm(F(x))])

    while (la.norm(F(x)) > tol):
        A = J(F, x)
        # use solver with/without preconditioner
        # TODO: When will this be singular?
        # TODO: Allow other methods like GMRES and maybe CGS
        s = spla.spsolve(A, -F(x))
        x = x + s
        niters += 1
        score_hist = np.append(score_hist, la.norm(F(x)))

    return x, niters, score_hist